In [ ]:

# train_and_save_model.ipynb

import yfinance as yf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import joblib
from datetime import datetime


# 1. Fetch stock data
def fetch_stock_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    data = data.dropna()
    return data

# 2. Example: Prepare the data (very simple example: predict "Close" from 1-day-lag)
def prepare_features_and_labels(data):
    # For simplicity, let's just shift the 'Close' price by 1 day to predict the next day's close
    data['Close_tomorrow'] = data['Close'].shift(-1)
    data.dropna(inplace=True)
    
    # X: today's close
    X = data[['Close']].values
    # y: tomorrow's close
    y = data['Close_tomorrow'].values
    return X, y

# Train function
def train_and_save_model(ticker):
    # Hyperparameters (tweak as needed)
    start_date = "2025-01-01"
    end_date = datetime.today().strftime("%Y-%m-%d")    
    # Fetch data
    stock_data = fetch_stock_data(ticker, start_date, end_date)
    
    # Prepare features/labels
    X, y = prepare_features_and_labels(stock_data)
    
    # Scale X
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Train XGBRegressor
    model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
    model.fit(X_scaled, y)
    
    # Evaluate quickly on the same data (just for demonstration)
    y_pred = model.predict(X_scaled)
    rmse = np.sqrt(mean_squared_error(y, y_pred))
    print(f"Trained model RMSE for {ticker}:", rmse)
    
    # Save the model and scaler
    joblib.dump(model, f"models/{ticker}_model.joblib")
    joblib.dump(scaler, f"models/{ticker}_scaler.joblib")

    # # Save the model and the scaler to the live folder.

    # joblib.dump(model, f"/Users/noel_personal/Repos/TIKR/my-react-app/backend/live_models/{ticker}_model.joblib")
    # joblib.dump(scaler, f"/Users/noel_personal/Repos/TIKR/my-react-app/backend/live_models/{ticker}_scaler.joblib")

    print(f"Model and scaler saved for ticker {ticker}.")

# Example usage: train and save for "AAPL"
train_and_save_model("AAPL")


[*********************100%***********************]  1 of 1 completed

Trained model RMSE for AAPL: 0.13685225104431117
Model and scaler saved for ticker AAPL.


In [2]:
import os 
os.getcwd()

'/Users/noel_personal/Repos/TIKR/my-react-app/ml'